In [ ]:
import ee
import geemap
import rasterio
import numpy as np
import glob
import os
from rasterio.merge import merge
import folium
from branca.colormap import LinearColormap
from datetime import datetime
import time

# Inicializar Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# Definir AOI para referência no mapa
aoi = ee.Geometry.Polygon([[[-51.163395, -18.742186], [-51.163395, -18.534308], 
                           [-50.957458, -18.534308], [-50.957458, -18.742186], 
                           [-51.163395, -18.742186]]])

# Diretório com os arquivos TIF
directory = 'analises_clorofila/cacu'

def get_unique_filename(base_path):
    """Gera um nome de arquivo único baseado em timestamp"""
    dirname = os.path.dirname(base_path)
    basename = os.path.basename(base_path)
    name, ext = os.path.splitext(basename)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    return os.path.join(dirname, f"{name}_{timestamp}{ext}")

def merge_tiff_files(directory):
    """Merge all TIFF files in the directory"""
    search_pattern = os.path.join(directory, "PredictedChlorophyll_Tile_*.tif")
    tiff_files = glob.glob(search_pattern)
    
    if not tiff_files:
        raise ValueError(f"No TIFF files found in {directory}")
    
    # Ler todos os arquivos TIF
    src_files_to_mosaic = []
    for tiff in tiff_files:
        try:
            src = rasterio.open(tiff)
            src_files_to_mosaic.append(src)
        except Exception as e:
            print(f"Erro ao abrir {tiff}: {str(e)}")
            continue
    
    if not src_files_to_mosaic:
        raise ValueError("Nenhum arquivo TIF válido encontrado")
    
    try:
        mosaic, out_trans = merge(src_files_to_mosaic)
        
        out_meta = src_files_to_mosaic[0].meta.copy()
        out_meta.update({
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_trans,
            "nodata": -9999  # Usar um valor numérico em vez de np.nan
        })
        
        # Gerar nome único para o arquivo mesclado
        base_merged_path = os.path.join(directory, "merged_chlorophyll.tif")
        merged_path = get_unique_filename(base_merged_path)
        
        # Tentar escrever o arquivo mesclado
        with rasterio.open(merged_path, "w", **out_meta) as dest:
            dest.write(mosaic)
        
        return merged_path
    
    finally:
        # Fechar todos os arquivos fonte
        for src in src_files_to_mosaic:
            src.close()

# Função para criar mapa do raster
def create_map_from_tiff(tiff_path):
    """Create map from TIFF file"""
    with rasterio.open(tiff_path) as src:
        data = src.read(1)
        bounds = [
            [src.bounds.bottom, src.bounds.left],
            [src.bounds.top, src.bounds.right]
        ]
        
        # Remover valores nodata antes de calcular min/max
        valid_data = data[data != -9999]
        valid_data = valid_data[~np.isnan(valid_data)]
        
        min_value = np.percentile(valid_data, 1)
        max_value = np.percentile(valid_data, 99)
        
        print(f"Valores de Clorofila - Min: {min_value:.2f}, Max: {max_value:.2f}")
        
        # Criar mapa base
        center_lat = (bounds[0][0] + bounds[1][0]) / 2
        center_lon = (bounds[0][1] + bounds[1][1]) / 2
        
        Map = geemap.Map()
        Map.centerObject(aoi, zoom=10)
        Map.add_basemap('SATELLITE')
        
        # Parâmetros de visualização
        vis_params = {
            'min': float(min_value),
            'max': float(max_value),
            'palette': [
                'blue', 'cyan', 'green', 'yellow', 'orange', 'red',  
                'darkred', 'purple', 'magenta'
            ],
            'opacity': 0.7
        }
        
        # Adicionar o raster
        Map.add_raster(
            tiff_path,
            vis_params=vis_params,
            layer_name='Predicted Chlorophyll'
        )
        
        # Adicionar contorno da AOI
        Map.addLayer(aoi, {'color': 'white', 'width': 2, 'fillColor': 'transparent'}, 'AOI Boundary')
        Map.addLayerControl()
        
        # Adicionar legenda
        def add_legend(map_obj, title, palette, min_value, max_value):
            legend_html = f"""
            <div style='padding: 10px; background-color: white; border-radius: 5px;'>
                <h4>{title}</h4>
                <div style='display: flex; align-items: center;'>
                    <span>{min_value:.2f}</span>
                    <div style='flex-grow: 1; height: 20px; background: linear-gradient(to right, {", ".join(palette)}); margin: 0 10px;'></div>
                    <span>{max_value:.2f}</span>
                </div>
            </div>
            """
            map_obj.add_html(legend_html)
        
        add_legend(Map, 'Predicted Chlorophyll', vis_params['palette'], min_value, max_value)
        
        return Map

try:
    # Mesclar os arquivos TIF
    merged_tiff = merge_tiff_files(directory)
    print(f"Arquivo mesclado criado em: {merged_tiff}")
    
    # Criar e mostrar o mapa
    Map = create_map_from_tiff(merged_tiff)
    Map

except Exception as e:
    print(f"Erro ao processar: {str(e)}")
    raise

Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_transparencia\cacu\PredictedTransparency_Tile_1_1.tif
Image saved locally: analises_transparencia/cacu\PredictedTransparency_Tile_1_1.tif
Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_transparencia\cacu\PredictedTransparency_Tile_1_2.tif
Image saved locally: analises_transparencia/cacu\PredictedTransparency_Tile_1_2.tif
Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_transparencia\cacu\PredictedTransparency_Tile_2_1.tif
Image saved locally: analises_transparencia/cacu\PredictedTransparency_Tile_2_1.tif
Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_transparencia\cacu\PredictedTransparency_Tile_2_2.tif
Image saved locally: analises_transparencia/cacu\PredictedTransparency_Tile_2_2.tif
Minimum Water Transparency Value: -1055.9283664319435
Maximum Water Transparency Value: 2024.8941273243227


Map(center=[-18.479438884362004, -51.18243850000118], controls=(WidgetControl(options=['position', 'transparen…